In [1]:
import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '')  # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text)  # заменяем все символы кроме кириллицы на пустые символы

In [4]:
# парсим текст, как последовательность символов
num_characters = 34  # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True)  # токенизируем на уровне символов
tokenizer.fit_on_texts([text])  # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index) # наша коллекция

{' ': 1, 'о': 2, 'е': 3, 'т': 4, 'и': 5, 'а': 6, 'н': 7, 'с': 8, 'в': 9, 'р': 10, 'м': 11, 'л': 12, 'ь': 13, 'д': 14, 'п': 15, 'у': 16, 'ы': 17, 'з': 18, 'я': 19, 'б': 20, 'ч': 21, 'к': 22, 'й': 23, 'ж': 24, 'г': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'щ': 30, 'э': 31, 'ф': 32, 'ъ': 33}


In [5]:
inp_chars = 6 # Делаем прогноз на основе N символов
data = tokenizer.texts_to_matrix(text)  # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars  # так как мы предсказываем по трем символам - четвертый

X = np.array([data[i:i + inp_chars, :] for i in range(n)]) # Тензор
Y = data[inp_chars:]  # предсказание следующего символа

print(data.shape)

(6307, 34)


In [6]:
model = Sequential()
model.add(Input((inp_chars,
                 num_characters)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 128 нейронов и фун-ия активация гиперболический тангенс
model.add(Dense(num_characters, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=100)

# Вспомогательная фун-ия для постройки фразы
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters) # формирование начальных долгов

        pred = model.predict(inp)  # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении

        inp_str += d  # дописываем строку

    return inp_str


res = buildPhrase("утренн")
print(res)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               20864     
                                                                 
 dense_1 (Dense)             (None, 34)                4386      
                                                                 
Total params: 25,250
Trainable params: 25,250
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
197/197 [==============================] - 1s 2ms/step - loss: 2.9779 - accuracy: 0.1811
Epoch 2/100
197/197 [==============================] - 0s 2ms/step - loss: 2.6440 - accuracy: 0.2557
Epoch 3/100
197/197 [==============================] - 0s 2ms/step - loss: 2.4540 - accuracy: 0.3095
Epoch 4/100
197/197 [==============================] - 0s 2ms/step - loss: 2.3233 - accuracy: 0.3360
Epoch 5/100
197/197 [=====================

197/197 [==============================] - 0s 2ms/step - loss: 0.4326 - accuracy: 0.8714
Epoch 75/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4240 - accuracy: 0.8683
Epoch 76/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8729
Epoch 77/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4157 - accuracy: 0.8748
Epoch 78/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4097 - accuracy: 0.8751
Epoch 79/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4039 - accuracy: 0.8762
Epoch 80/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4087 - accuracy: 0.8786
Epoch 81/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3952 - accuracy: 0.8789
Epoch 82/100
197/197 [==============================] - 0s 2ms/step - loss: 0.4015 - accuracy: 0.8762
Epoch 83/100
197/197 [==============================] - 0s 2ms/step - loss: 0.3895 - accuracy: 